In [3]:
from collections import defaultdict

class TreeNode:
    def __init__(self, item):
        self.item = item
        self.count = 1
        self.parent = None
        self.children = {}
        self.link = None  

def build_tree(transactions, min_support):
    """ Build the FP-tree and header table. """
    item_counts = defaultdict(int)

    for transaction in transactions:
        for item in transaction:
            item_counts[item] += 1

    item_counts = {k: v for k, v in item_counts.items() if v >= min_support}
    if not item_counts:
        return None, None

    header_table = {item: None for item in item_counts}

    root = TreeNode(None) 
    for transaction in transactions:
        sorted_items = [item for item in sorted(transaction, key=lambda x: -item_counts.get(x, 0)) if item in item_counts]
        insert_tree(sorted_items, root, header_table)

    return root, header_table

def insert_tree(items, node, header_table):
    """ Insert items into the FP-tree. """
    if not items:
        return
    
    first_item = items[0]
    if first_item in node.children:
        node.children[first_item].count += 1
    else:
        new_node = TreeNode(first_item)
        new_node.parent = node
        node.children[first_item] = new_node

        if header_table[first_item] is None:
            header_table[first_item] = new_node
        else:
            link_nodes(header_table[first_item], new_node)

    insert_tree(items[1:], node.children[first_item], header_table)

def link_nodes(start_node, new_node):
    """ Link similar nodes together in the header table. """
    while start_node.link:
        start_node = start_node.link
    start_node.link = new_node

def find_frequent_patterns(header_table, min_support):
    """ Extract frequent patterns from the FP-tree. """
    patterns = {}
    for item in header_table:
        count = 0
        node = header_table[item]

        while node:
            count += node.count
            node = node.link

        if count >= min_support:
            patterns[item] = count

    return patterns

transactions = [
    ['A', 'B', 'D'],
    ['B', 'C', 'E'],
    ['A', 'B', 'C', 'E'],
    ['A', 'C', 'D', 'E'],
    ['B', 'C']
]

min_support = 2  

root, header_table = build_tree(transactions, min_support)
if root:
    frequent_patterns = find_frequent_patterns(header_table, min_support)
    print("Frequent Patterns:", frequent_patterns)
else:
    print("No frequent patterns found.")


Frequent Patterns: {'A': 3, 'B': 4, 'D': 2, 'C': 4, 'E': 3}
